<a href="https://colab.research.google.com/github/GinuraAdikari/InsightHive/blob/Sentiment_Analysis/Model_ABSA_DataSet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
import pandas as pd
import numpy as np

ABSA = pd.read_csv("ABSA_dataset.csv")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

# train (80%) and test (20%)
train_df, test_df = train_test_split(ABSA, test_size=0.2, random_state=42, stratify=ABSA["sentiment"])

print("Train and test datasets created!")

Train and test datasets created!


Input Representation for Aspect Awareness

In [ ]:
def convert_example_to_features(sentence, aspect, tokenizer, max_length=128):
    """
    Converts a sentence into BERT input features, including an aspect mask.

    - sentence: Full review text
    - aspect: Aspect term (e.g., 'food', 'service')
    - tokenizer: BERT tokenizer
    - max_length: Maximum sequence length
    """
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Create attention mask
    attention_mask = [1] * len(input_ids)

    # Generate aspect mask (1 for aspect-related tokens, 0 otherwise)
    aspect_mask = [0] * len(input_ids)
    aspect_tokens = tokenizer.tokenize(aspect)

    for i in range(len(tokens) - len(aspect_tokens) + 1):
        if tokens[i: i + len(aspect_tokens)] == aspect_tokens:
            aspect_mask[i: i + len(aspect_tokens)] = [1] * len(aspect_tokens)

    # Pad sequences
    padding_length = max_length - len(input_ids)
    input_ids += [0] * padding_length
    attention_mask += [0] * padding_length
    aspect_mask += [0] * padding_length

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long)
    }


BERT model for Aspect-Aware Attention

In [ ]:
from transformers import BertPreTrainedModel, BertModel
import torch
import torch.nn as nn

class BertForABSA(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.aspect_attention = nn.Linear(config.hidden_size, 1)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, attention_mask, aspect_mask, labels=None):
        # Get BERT output
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_dim)

        # Apply aspect-aware attention
        aspect_weights = torch.sigmoid(self.aspect_attention(hidden_states))  # Compute attention scores
        aspect_weights = aspect_weights * aspect_mask.unsqueeze(-1)  # Apply aspect mask
        aspect_representations = hidden_states * aspect_weights  # Scale hidden states by attention weights

        # Pool aspect-aware representations
        pooled_output = aspect_representations.mean(dim=1)

        # Classification layer
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # Compute loss
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return loss, logits


Training pipeline

In [ ]:
from torch.utils.data import Dataset
import torch

class ABSADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokens = df["tokens"].tolist()
        self.aspect_masks = df["aspect_mask"].tolist()
        self.sentiments = df["sentiment"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]

        # Ensure tokens is a list
        if isinstance(tokens, str):  # If stored as string, convert to list
            tokens = eval(tokens)
        elif not isinstance(tokens, list):  # If somehow still not list, wrap in list
            tokens = [tokens]

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # Ensure aspect_mask is a list
        aspect_mask = self.aspect_masks[idx]
        if isinstance(aspect_mask, str):
            aspect_mask = eval(aspect_mask)
        elif not isinstance(aspect_mask, list):
            aspect_mask = [aspect_mask]

        # Create attention mask (1 for actual tokens, 0 for padding)
        attention_mask = [1] * len(input_ids)

        # **🔹 Fix: Ensure consistent sequence length using padding**
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length]
            attention_mask = attention_mask[:self.max_length]
            aspect_mask = aspect_mask[:self.max_length]
        else:
            padding_length = self.max_length - len(input_ids)
            input_ids += [0] * padding_length
            attention_mask += [0] * padding_length
            aspect_mask += [0] * padding_length

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long),
            "labels": torch.tensor(self.sentiments[idx], dtype=torch.long)
        }

print("✅ Dataset class loaded successfully!")


✅ Dataset class loaded successfully!


class imbalance

In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

train_dataset = ABSADataset(train_df, tokenizer)
test_dataset = ABSADataset(test_df, tokenizer)

# ✅ Fix: Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Fix: Ensure `y` is a NumPy array
class_labels = np.array([0, 1])  # Only two classes now
class_weights = compute_class_weight("balanced", classes=class_labels, y=train_df["sentiment"].values)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Train and Test DataLoaders are ready!")

Train and Test DataLoaders are ready!


Train

In [ ]:
from transformers import AdamW, get_scheduler
import torch

# Initialize model
model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=2)  # 3 sentiment classes (Neg, Neu, Pos)

# Define optimizer with weight decay for better generalization
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

num_epochs = 3

# Define learning rate scheduler
num_training_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
)

# Set up device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
print("🚀 Starting Training...")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    num_batches = len(train_dataloader)

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        loss, logits = model(
            batch["input_ids"],
            batch["attention_mask"],
            batch["aspect_mask"],
            batch["labels"]
        )

        loss = loss_fn(logits, batch["labels"])

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # Print loss every 50 steps
        if (step + 1) % 50 == 0 or (step + 1) == num_batches:
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{step+1}/{num_batches}] - Loss: {loss.item():.4f}")

    avg_loss = total_loss / num_batches
    print(f"✅ Epoch {epoch+1} Completed! Avg Loss: {avg_loss:.4f}")

print("🎉 Training Finished!")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


🚀 Starting Training...
Epoch [1/3] Step [50/66] - Loss: 0.6844
Epoch [1/3] Step [66/66] - Loss: 0.7161
✅ Epoch 1 Completed! Avg Loss: 0.6916
Epoch [2/3] Step [50/66] - Loss: 0.7050
Epoch [2/3] Step [66/66] - Loss: 0.6266
✅ Epoch 2 Completed! Avg Loss: 0.6270
Epoch [3/3] Step [50/66] - Loss: 0.4958
Epoch [3/3] Step [66/66] - Loss: 0.4560
✅ Epoch 3 Completed! Avg Loss: 0.4771
🎉 Training Finished!


Evaluate

In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(model, dataloader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(batch["labels"].cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

# Run evaluation
evaluate_model(model, test_dataloader)


              precision    recall  f1-score   support

    Negative       0.82      0.66      0.73        82
    Positive       0.86      0.93      0.89       180

    accuracy                           0.85       262
   macro avg       0.84      0.80      0.81       262
weighted avg       0.84      0.85      0.84       262



hyperparameter tunning

In [ ]:
import torch
import torch.optim as optim
from transformers import get_scheduler
from sklearn.model_selection import ParameterGrid
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# Define possible hyperparameters
param_grid = {
    "learning_rate": [2e-5, 3e-5, 5e-5],  # Common BERT fine-tuning rates
    "batch_size": [8, 16],  # Trade-off between speed and performance
    "weight_decay": [0.01, 0.001]  # Regularization to avoid overfitting
}

# Create all possible hyperparameter combinations
param_combinations = list(ParameterGrid(param_grid))

best_model = None
best_f1_score = 0  # Track the best model based on F1-score

for params in param_combinations:
    print(f"\n🚀 Training with params: {params}")

    # Load model
    model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=2)
    model.to(device)

    # Optimizer & Learning Rate Scheduler
    optimizer = optim.AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
    num_training_steps = len(train_dataloader) * 3  # Assuming 3 epochs
    lr_scheduler = get_scheduler(
        "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
    )

    # Loss function
    criterion = torch.nn.CrossEntropyLoss()

    # Data Loaders
    train_dataloader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    val_dataloader = DataLoader(test_dataset, batch_size=params["batch_size"])

    # Training Loop
    for epoch in range(3):
        model.train()
        total_loss = 0
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()

            loss, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"], batch["labels"])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            lr_scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} - Loss: {total_loss / len(train_dataloader)}")

    # Validate Model
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(batch["labels"].cpu().numpy())

    # Compute F1-score
    report = classification_report(y_true, y_pred, target_names=["Negative", "Positive"], output_dict=True)
    f1_score = report["macro avg"]["f1-score"]
    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

    # Save best model based on F1-score
    if f1_score > best_f1_score:
        best_f1_score = f1_score
        best_model = model
        best_params = params

# Save the best model
best_model.save_pretrained("best_absa_model")
print(f"✅ Best model saved with params: {best_params} (F1-score: {best_f1_score:.4f})")


🚀 Training with params: {'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.01}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.6534200262932377
Epoch 2 - Loss: 0.5424236830409247
Epoch 3 - Loss: 0.5091745596350604
              precision    recall  f1-score   support

    Negative       0.70      0.39      0.50        82
    Positive       0.77      0.92      0.84       180

    accuracy                           0.76       262
   macro avg       0.73      0.66      0.67       262
weighted avg       0.75      0.76      0.73       262


🚀 Training with params: {'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.001}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.6641075499640167
Epoch 2 - Loss: 0.5481806626074187
Epoch 3 - Loss: 0.395243507873921
              precision    recall  f1-score   support

    Negative       0.80      0.65      0.72        82
    Positive       0.85      0.93      0.89       180

    accuracy                           0.84       262
   macro avg       0.83      0.79      0.80       262
weighted avg       0.84      0.84      0.83       262


🚀 Training with params: {'batch_size': 8, 'learning_rate': 3e-05, 'weight_decay': 0.01}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
